In [37]:
import torch
from torch import nn
from torchvision import datasets, transforms
import torch.nn.functional as F

import os

In [31]:
!unzip -q /kaggle/input/generative-dog-images/all-dogs.zip -d /kaggle/working/all-dogs/

In [32]:
image_files = os.listdir('/kaggle/working/all-dogs')
print(f'Number of images in the dataset = {len(image_files)}')

Number of images in the dataset = 20580


In [33]:
PATH = '/kaggle/working/all-dogs'
BATCH_SIZE = 32

In [34]:
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [35]:
train_data = datasets.ImageFolder(PATH, transform=transforms)
train_dataloader = torch.utils.data.DataLoader(train_data, 
                                               batch_size=BATCH_SIZE,
                                               shuffle=True)

In [39]:
class Generator(nn.Module):
    
    def __init__(self, n_z, n_feats, n_channels):
        super(Generator, self).__init__()
        
        self.conv1 = nn.ConvTranspose2d(in_channels=n_z, 
                                        out_channels=n_feats * 8, 
                                        kernel_size=4, 
                                        stride=1, 
                                        padding=0, 
                                        bias=False)
        self.bn1 = nn.BatchNorm2d(num_feastures=n_feats * 8)
        
        self.conv2 = nn.ConvTranspose2d(in_channels=n_feats * 8,
                                        out_channels=n_feats * 8,
                                        kernel_size=4, 
                                        stride=2,
                                        padding=1,
                                        bias=False)
        self.bn2 = nn.BatchNorm2d(num_features=n_feats * 8)
        
        self.conv3 = nn.ConvTranspose2d(in_channels=n_feats * 8, 
                                        out_channels=n_feats * 4,
                                        kernel_size=4,
                                        stride=2,
                                        padding=1,
                                        bias=False)
        self.bn3 = nn.BatchNorm2d(num_features=n_feats * 4)
        
        self.conv4 = nn.ConvTranspose2d(in_channels=n_feats * 4,
                                        out_channels=n_feats * 2,
                                        kernel_size=4,
                                        stride=2,
                                        padding=1,
                                        bias=False)
        self.bn4 = nn.BatchNorm2d(num_features=n_feats * 2)
        
        self.conv5 = nn.ConvTranspose2d(in_channels=n_feats * 2,
                                        out_channels=n_feats,
                                        kernel_size=4,
                                        stride=2,
                                        padding=1, 
                                        bias=False)
        self.bn5 = nn.BatchNorm2d(num_features=n_feats)
        
        self.conv6 = nn.ConvTranspose2d(in_channels=n_feats,
                                        out_channels=n_channels,
                                        kernel_size=3,
                                        stride=1,
                                        padding=1,
                                        bias=False)
        
    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.conv1(x)))
        x = F.leaky_relu(self.bn2(self.conv2(x)))
        x = F.leaky_relu(self.bn3(self.conv3(x)))
        x = F.leaky_relu(self.bn4(self.conv4(x)))
        x = F.leaky_relu(self.bn5(self.conv5(x)))
        x = torch.tanh(self.conv6(x))
        
        return x